# Drops of Jupyter:  Making Maps with Python and Interactive Notebooks

**Ryan Cooper |  @maptastik**

Technology Analyst | Raleigh Parks, Recreation and Cultural Resources

**Presentation Repo:** https://github.com/maptastik/gisday_2019

1. What is a Jupyter Notebook?
2. Static Maps
3. Interactive Maps
4. Advanced Applications

## What is a Jupyter Notebook?

_The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text._ - [jupyter.org](https://jupyter.org/)

- Python
- R
- Julia
- and over 100 other languages

# THIS.

## Pandas + Shapely: A standard data model

In [ ]:
import geopandas as gpd

### Read a file

In [ ]:
parks_gdf = gpd.read_file('https://opendata.arcgis.com/datasets/43b5d6bf9d6e400599498d052545d331_0.geojson')

### View the data frame

In [ ]:
parks_gdf.head()

### Get information about the dataset

#### Fields

In [ ]:
parks_gdf.info()

#### Summay Statistics

In [ ]:
parks_gdf.describe()

In [ ]:
parks_gdf['DEVELOPED'].value_counts()

#### Geometry Properties

In [ ]:
parks_gdf.crs

In [ ]:
park_geometry = parks_gdf.loc[parks_gdf[parks_gdf["NAME"] == 'Brentwood'].index[0], 'geometry']
display(park_geometry)
print(f'Python Object Type: {type(park_geometry)}')
print(f'Geometry Type: {park_geometry.geom_type}')

In [ ]:
print(park_geometry)

#### Shapely Geometry

In [ ]:
park_geometry.centroid.x, park_geometry.centroid.y

In [ ]:
from pyproj import Proj, transform
inProj = Proj(init = 'epsg:4326')
outProj = Proj(init = 'epsg:2264')
x, y = transform(inProj, outProj, park_geometry.centroid.x, park_geometry.centroid.y)
x, y

In [ ]:
parks_points_gdf = parks_gdf.copy()
parks_points_gdf['geometry'] = parks_points_gdf.apply(lambda x: x['geometry'].centroid, axis = 1)
parks_points_gdf.plot()

## Static Maps

#### Plot geometries with geopandas

In [ ]:
%matplotlib inline
parks_gdf.plot(column = 'DEVELOPED', cmap = 'Paired', figsize = (12, 12), legend = True)

### Example: Citrix Cycle Docks w/ geopandas and contextily

In [ ]:
import requests
from io import BytesIO
import pandas as pd
from shapely.geometry import Point

#### Access Citrix Cycle Docks data feed

In [ ]:
cc_docks_r = requests.get('https://citrixcycle.com/stations/stations/')
cc_docks_df = pd.read_json(BytesIO(cc_docks_r.content))
cc_docks_reduced_df = cc_docks_df.loc[:,['id', 'locking_station_type', 'description', 'address', 'primary_locked_cycle_count', 'stocking_low', 'total_locked_cycle_count', 'free_spaces', 'location']]
cc_docks_reduced_df['geometry'] = cc_docks_reduced_df.apply(lambda x: Point(x['location'][1],x['location'][0]), axis = 1)
cc_docks_reduced_df.drop(columns=['location'], inplace = True)
cc_docks_reduced_gdf = gpd.GeoDataFrame(cc_docks_reduced_df, geometry = 'geometry', crs={'init': 'epsg:4326'})
cc_docks_reduced_gdf.head()

In [ ]:
cc_docks_reduced_gdf.plot()

#### Plot with contextily

In [ ]:
import contextily as ctx

In [ ]:
cc_ax1 = cc_docks_reduced_gdf.to_crs(epsg = 3857).plot(figsize = (12, 20))
ctx.add_basemap(cc_ax1, url = 'https://a.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png')

#### Size maker by column value

In [ ]:
cc_docks_reduced_gdf.describe()

In [ ]:
cc_ax2 =cc_docks_reduced_gdf.to_crs(epsg = 3857).plot(markersize = cc_docks_reduced_gdf['total_locked_cycle_count']**3, figsize = (12, 20))
ctx.add_basemap(cc_ax2, url = 'https://a.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png')

In [ ]:
cc_ax3 = cc_docks_reduced_gdf.to_crs(epsg = 3857).plot(
    marker = 'o',
    markersize = cc_docks_reduced_gdf['total_locked_cycle_count']**3,
    color = 'purple',
    edgecolors = 'black',
    linewidths = 3,
    alpha = 0.35,
    figsize = (12,20)) 
ctx.add_basemap(cc_ax3, url = 'https://a.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png')

### Example: Plotting building permit density with geoplot

#### Access last month of Raleigh building permit data

In [ ]:
bp_gdf = gpd.read_file('https://opendata.arcgis.com/datasets/f7a3cbd07e9f4ca5bb6637e7eeab5871_0.geojson')
# bp_gdf = gpd.read_file('./DATA/building_permits_1month.geojson')
bp_gdf.drop(bp_gdf[bp_gdf['geometry'].isna()].index, inplace = True)
bp_gdf.head()

#### Plotting building permit density with geoplot

In [ ]:
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt

In [ ]:
bp_ax1 = gplt.pointplot(bp_gdf, projection = gcrs.WebMercator(), s = 8, color = 'green', figsize = (12, 12))
gplt.webmap(bp_gdf, provider = 'ST_TONER', ax = bp_ax1)

In [ ]:
bp_ax2 = gplt.kdeplot(bp_gdf, projection = gcrs.WebMercator(), cmap = 'plasma', shade = True, alpha = 0.5, figsize = (12, 12))
gplt.pointplot(bp_gdf, s = 2, color = 'black', ax = bp_ax2)
gplt.webmap(bp_gdf, provider = 'ST_TONER', ax = bp_ax2)
plt.title("Density of Raleigh Building Permits for the last month", fontsize = 24)
fig = plt.gcf()

## Interactive

### Example: Interactive Choropleth

First, a data processing diversion...Aggregating points to a hexgrid!

In [ ]:
hexgrid_gdf = gpd.read_file('./DATA/hexgrid_polygon_4326.geojson')
hex_ax = gplt.polyplot(hexgrid_gdf, projection = gcrs.WebMercator(), color = 'red', alpha = 0.25, figsize = (12, 12), zorder = 1)
gplt.pointplot(bp_gdf, s = 2, color = 'black', ax = hex_ax, zorder = 2)
gplt.webmap(hexgrid_gdf, provider = 'ST_TONER', ax = hex_ax)

In [ ]:
bp_hex_counts_df = gpd.sjoin(hexgrid_gdf, bp_gdf)['hexid'].value_counts().reset_index().rename(columns = {'index': 'hexid', 'hexid': 'bp_count'})
bp_hex_counts_gdf = hexgrid_gdf.merge(bp_hex_counts_df, on = 'hexid')
bp_hex_counts_gdf.head()

#### Folium

In [ ]:
import folium

In [ ]:
m1 = folium.Map(location = [35.84, -78.638176], zoom_start = 12)
m1

In [ ]:
m2 = folium.Map(
    location = [35.84, -78.638176], 
    zoom_start = 12,
    tiles = 'stamentoner')
folium.Choropleth(
    geo_data = './DATA/hexgrid_polygon_4326.geojson',
    name = 'Building Permit Hexbins',
    data = bp_hex_counts_df,
    columns = ['hexid', 'bp_count'],
    key_on = 'feature.properties.hexid',
    fill_color = 'Greens',
    fill_opacity = 0.75,
    highlight = True,
    nan_fill_opacity = 0,
    nan_fill_color = '#ffffff',
    line_weight = 0,
    control = True
).add_to(m2)

folium.LayerControl().add_to(m2)
m2

#### CARTOframes

In [ ]:
from cartoframes.viz.helpers import color_continuous_layer
color_continuous_layer(bp_hex_counts_gdf, 'bp_count', 'Building Permits')

In [ ]:
Map(
    Layer(
        bp_hex_counts_gdf,
        '''
        color: ramp(globalQuantiles($bp_count,8), teal)
        opacity: 0.75
        ''',
        legend = Legend(
            'color-bins',
            title = 'Building Permits',
            description = "by hex grid cell",
            footer = "City of Raleigh GIS"
        ),
        popup = Popup({
            'hover': {'title': 'Total Building Permits', 'value':'$bp_count'}
        })
    )
)

### Advanced Applications

But first, some combining GoRaleigh Stop and Shelter datasets...

In [ ]:
def arcgis_rest_to_gdf(url, layer_id):
  url = f'{url}/{layer_id}/query'
  params = {
    'f': 'geojson',
    'where': '1=1',
    'outFields': '*',
    'outSR': 4326
  }
  r = requests.get(url, params = params)
  return gpd.read_file(BytesIO(r.content))

shelters_gdf = arcgis_rest_to_gdf("https://services.arcgis.com/v400IkDOw1ad7Yad/arcgis/rest/services/GoRaleigh_Shelters/FeatureServer", 0)
stops_gdf = arcgis_rest_to_gdf("https://services.arcgis.com/v400IkDOw1ad7Yad/ArcGIS/rest/services/GoRaleigh_Stops/FeatureServer", 0)

shelters_reduced_gdf = shelters_gdf.loc[:,["Stop_ID", "Stop_Name", "Shelter", "geometry"]]
shelters_reduced_gdf["Stop_ID"] = shelters_reduced_gdf.apply(lambda x: str(x["Stop_ID"]), axis = 1)
shelters_reduced_gdf["Status"] = shelters_reduced_gdf.apply(lambda x: "Planned" if "Planned" in x["Shelter"] else "Existing", axis = 1)
shelters_reduced_gdf["Shelter"] = shelters_reduced_gdf.apply(lambda x: x["Shelter"].split(' - ')[0], axis = 1)

stops_reduced_gdf = stops_gdf[["StopAbbr", "StopName", "geometry"]]
stops_reduced_gdf = stops_reduced_gdf.groupby("StopAbbr").first().reset_index()[["StopAbbr", "StopName", "geometry"]]

stops_shelters_gdf = stops_reduced_gdf.merge(shelters_reduced_gdf, how = 'outer', left_on = 'StopAbbr', right_on = 'Stop_ID', suffixes = ('', '_shelters'), sort = True)

# Clean up some of the fields and pivot Status field for use with formula widget
stops_shelters_gdf['StopAbbr'].fillna('0', inplace = True)
stops_shelters_gdf["StopName"].fillna("Unnamed Stop", inplace = True)
stops_shelters_gdf["Shelter"].fillna("No Shelter", inplace = True)
stops_shelters_gdf["Status"].fillna('No Shelter Planned', inplace = True)
stops_shelters_gdf["Existing"] = stops_shelters_gdf.apply(lambda x: 1 if x["Status"] == "Existing" else 0, axis = 1)
stops_shelters_gdf["Planned"] = stops_shelters_gdf.apply(lambda x: 1 if x["Status"] == "Planned" else 0, axis = 1)
stops_shelters_gdf["No_Shelter_Planned"] = stops_shelters_gdf.apply(lambda x: 1 if x["Status"] == "No Shelter Planned" else 0, axis = 1)
stops_shelters_gdf["geometry"] = stops_shelters_gdf.apply(lambda x: x["geometry_shelters"] if x["geometry"] is None else x["geometry"], axis = 1)

stops_shelters_gdf = stops_shelters_gdf[["StopAbbr", "StopName", "Shelter", "Status", "Existing", "Planned", "No_Shelter_Planned", "geometry"]]
stops_shelters_gdf = gpd.GeoDataFrame(stops_shelters_gdf, crs = {"init":"epsg:4326"}, geometry = "geometry")

In [ ]:
stops_shelters_gdf.head()

#### CARTOframes

In [ ]:
from cartoframes.viz import Map, Layer, Legend, Popup
from cartoframes.viz.widgets import category_widget, formula_widget

In [ ]:
shelter_stop_map = Map(
    Layer(
        stops_shelters_gdf,
        '''
        color: ramp(buckets($Status, ["Existing", "Planned", "No Shelter Planned"]), [#4CAF50, #FFC107, #B0BEC533]),
        width: 5
        ''',
        legend = Legend(
            'color-category',
            title = "GoRaleigh Status of Bus Stop Shelters",
            footer = "Data: GoRaleigh, City of Raleigh"
        ),
        popup = Popup({
           'click': [{
                'title': 'Stop',
                'value':'$StopName'
               }, {
                'title': 'Shelter',
                'value': '$Shelter'
               }, {
                'title': 'Shelter Status',
                'value': '$Status'
               }
            ] 
        }),
        widgets = [
          formula_widget(
              'Existing',
              'sum',
              title = "Existing Bus Stop Shelters"
          ),
          formula_widget(
              'Planned',
              'sum',
              title = "Planned Bus Stop Shelters"
          ),
          formula_widget(
              'No_Shelter_Planned',
              'sum',
              title = "No Planned Bus Stop Shelters"
          ),
          category_widget(
              'Status',
              title = "Shelter Status",
              description = "Click to filter by shelter status"
          ),
          category_widget(
              'Shelter',
              title = "Shelter Type",
              description = "Click to filter by shelter Type"
          )
        ]
    )
)

In [ ]:
shelter_stop_map

#### Kepler

In [ ]:
from keplergl import KeplerGl

Before using Kepler, we'll do some data maniuplation to separate the bus stop coordinates into `x` and `y` columns.

In [ ]:
stops_shelters_xy_gdf = stops_shelters_gdf
stops_shelters_xy_gdf['x'] = stops_shelters_xy_gdf.apply(lambda x: x['geometry'].x, axis = 1)
stops_shelters_xy_gdf['y'] = stops_shelters_xy_gdf.apply(lambda x: x['geometry'].y, axis = 1)
stops_shelters_xy_gdf.head()

#### Bus Stop & Shelters

In [ ]:
map_1 = KeplerGl(
    height = 1000,
    data = {
        "bus": stops_shelters_xy_gdf
    }
)
map_1

#### Census Block - Park Connections

In [ ]:
ebpa_df = pd.read_csv('./DATA/ebpa_all_los_origin_dest_201902.csv')
map_2_config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['ebpa'],
     'id': 'rarxykf9',
     'name': ['analysis_class'],
     'type': 'range',
     'value': [0.5, 4],
     'enlarged': False,
     'plotType': 'histogram',
     'yAxis': None},
    {'dataId': ['ebpa'],
     'id': 'nbg2l08d7n',
     'name': ['total_length'],
     'type': 'range',
     'value': [0, 1000],
     'enlarged': False,
     'plotType': 'histogram',
     'yAxis': None}],
   'layers': [{'id': 'deygtuk',
     'type': 'arc',
     'config': {'dataId': 'ebpa',
      'label': 'ebpa',
      'color': [206, 64, 170],
      'columns': {'lat0': 'origin_y',
       'lng0': 'origin_x',
       'lat1': 'destination_y',
       'lng1': 'destination_x'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [117, 222, 227]},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': '4b4gt44',
     'type': 'geojson',
     'config': {'dataId': 'parks',
      'label': 'parks',
      'color': [77, 193, 156],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [119, 110, 87],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked': True,
       'filled': True,
       'enable3d': False,
       'wireframe': False},
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'ebpa': ['origin_id',
       'origin_x',
       'origin_y',
       'parkid',
       'destination_id'],
      'parks': ['OBJECTID',
       'NAME',
       'PARK_TYPE',
       'FILE_NUMBER',
       'PARCEL_COUNT']},
     'enabled': False},
    'brush': {'size': 0.5, 'enabled': True}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': 24,
   'dragRotate': True,
   'latitude': 35.78332321445701,
   'longitude': -78.65892203437697,
   'pitch': 50,
   'zoom': 13.277939926587665,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': False,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

In [ ]:
map_2 = KeplerGl(
    height = 600,
    data = {
        "ebpa": ebpa_df,
        "parks": parks_gdf
    }
)
map_2.config = map_2_config
map_2

In [ ]:
map_2.config